In [1]:
import tensorflow as tf
import os
import zipfile
from os import path, getcwd, chdir
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
path = f"{getcwd()}/dataset-2.zip"

In [3]:
zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

In [4]:
train = pd.read_csv('train.csv')

In [5]:
train[0:][0:]

,Image,target
0,96.jpg,manipuri
1,163.jpg,bharatanatyam
2,450.jpg,odissi
3,219.jpg,kathakali
4,455.jpg,odissi
...,...,...
359,72.jpg,manipuri
360,253.jpg,mohiniyattam
361,404.jpg,kuchipudi
362,89.jpg,manipuri


In [6]:
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img('downloads/dataset-2/train/'+train['Image'][i], target_size=(28,28,1), grayscale=True)
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

  0%|          | 0/364 [00:00<?, ?it/s]/opt/anaconda3/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:104: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
100%|██████████| 364/364 [00:03<00:00, 97.50it/s] 


In [7]:
for i in range(train.shape[0]):
    if(train['target'][i]=='manipuri'):
        train['target'][i]=1
    elif(train['target'][i]=='bharatanatyam'):
        train['target'][i]=2
    elif(train['target'][i]=='odissi'):
        train['target'][i]=3
    elif(train['target'][i]=='kathakali'):
        train['target'][i]=4
    elif(train['target'][i]=='sattriya'):
        train['target'][i]=5
    elif(train['target'][i]=='kathak'):
        train['target'][i]=6
    elif(train['target'][i]=='mohiniyattam'):
        train['target'][i]=7
    elif(train['target'][i]=='kuchipudi'):
        train['target'][i]=8
    else:
        train['target'][i]=9

In [8]:
y=train['target'].values
y = to_categorical(y)

In [ ]:
#noise = np.random.random((IMG_SIZE, IMG_SIZE), np.uint8)
#Matrix addition.. add noise to the image
#image += noise

In [9]:
class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get('acc')>0.96):
                print("\nReached 96% accuracy so cancelling training!")
                self.model.stop_training = True
callbacks = myCallback()

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [14]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28,28,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(9, activation='softmax'))

In [15]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['acc'])

In [16]:
model.fit(X_train, y_train, epochs=40, validation_data=(X_test, y_test),callbacks=[callbacks])

Train on 291 samples, validate on 73 samples
Epoch 1/40
291/291 [==============================] - 3s 10ms/step - loss: 2.1655 - acc: 0.1168 - val_loss: 2.1116 - val_acc: 0.1918
Epoch 2/40
291/291 [==============================] - 1s 3ms/step - loss: 2.0564 - acc: 0.1821 - val_loss: 2.0237 - val_acc: 0.1781
Epoch 3/40
291/291 [==============================] - 1s 3ms/step - loss: 2.0151 - acc: 0.2302 - val_loss: 1.9999 - val_acc: 0.2740
Epoch 4/40
291/291 [==============================] - 1s 3ms/step - loss: 1.8514 - acc: 0.3402 - val_loss: 1.9150 - val_acc: 0.2329
Epoch 5/40
291/291 [==============================] - 1s 3ms/step - loss: 1.7561 - acc: 0.3643 - val_loss: 1.8824 - val_acc: 0.3425
Epoch 6/40
291/291 [==============================] - 1s 3ms/step - loss: 1.5947 - acc: 0.4845 - val_loss: 1.8115 - val_acc: 0.3699
Epoch 7/40
291/291 [==============================] - 1s 4ms/step - loss: 1.3922 - acc: 0.5533 - val_loss: 1.7903 - val_acc: 0.4384
Epoch 8/40
291/291 [==========

In [17]:
test_file = pd.read_csv('test.csv')

In [18]:
test_file

,Image
0,508.jpg
1,246.jpg
2,473.jpg
3,485.jpg
4,128.jpg
...,...
151,366.jpg
152,226.jpg
153,35.jpg
154,458.jpg


In [19]:
test_image = []
for i in tqdm(range(test_file.shape[0])):
    img = image.load_img('downloads/dataset-2/test/'+test_file['Image'][i], target_size=(28,28,1), grayscale=True)
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test = np.array(test_image)

  0%|          | 0/156 [00:00<?, ?it/s]/opt/anaconda3/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:104: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
100%|██████████| 156/156 [00:01<00:00, 88.17it/s] 


In [20]:
prediction = model.predict_classes(test)

In [21]:
prediction

array([3, 5, 3, 3, 6, 8, 1, 8, 5, 8, 7, 2, 5, 5, 5, 2, 7, 2, 7, 3, 8, 7,
       4, 7, 1, 4, 7, 2, 4, 2, 5, 3, 1, 2, 7, 7, 3, 3, 1, 8, 6, 6, 3, 7,
       5, 7, 8, 5, 4, 8, 7, 7, 6, 5, 4, 8, 8, 3, 8, 5, 4, 5, 6, 7, 8, 8,
       4, 2, 4, 8, 6, 2, 7, 7, 6, 4, 1, 5, 7, 1, 7, 8, 7, 8, 8, 4, 5, 8,
       1, 7, 1, 7, 6, 8, 7, 6, 6, 5, 3, 7, 8, 4, 4, 5, 2, 8, 4, 5, 1, 3,
       5, 4, 8, 3, 8, 4, 7, 4, 7, 8, 3, 8, 8, 7, 3, 7, 5, 5, 3, 3, 6, 5,
       2, 4, 5, 5, 3, 1, 3, 6, 5, 6, 8, 4, 8, 4, 8, 5, 1, 3, 5, 5, 8, 6,
       3, 5])

In [22]:
len(prediction)

156

In [23]:
lst=[]
for i in range(156):
    if(prediction[i]==1):
        lst.append('manipuri')
    elif(prediction[i]==2):
        lst.append('bharatanatyam')
    elif(prediction[i]==3):
        lst.append('odissi')
    elif(prediction[i]==4):
        lst.append('kathakali')
    elif(prediction[i]==5):
        lst.append('sattriya')
    elif(prediction[i]==6):
        lst.append('kathak')
    elif(prediction[i]==7):
        lst.append('mohiniyattam')
    elif(prediction[i]==8):
        lst.append('kuchipudi')
    else:
        lst.append('kathak')

In [24]:
#df = pd.DataFrame(lst)

In [25]:
import pandas
df = pandas.DataFrame(data={"Image": test_file['Image'], "target": lst})
df.to_csv("./file.csv", sep=',',index=False)

In [26]:
df.to_csv('filename.csv', index=False)

In [27]:
df

,Image,target
0,508.jpg,odissi
1,246.jpg,sattriya
2,473.jpg,odissi
3,485.jpg,odissi
4,128.jpg,kathak
...,...,...
151,366.jpg,sattriya
152,226.jpg,kuchipudi
153,35.jpg,kathak
154,458.jpg,odissi


In [28]:
lst.count(None)

0